In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.trend import AroonIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
#from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator
import time
from IPython.display import clear_output
from datetime import datetime
from scipy import stats

start=(datetime.now())

sip500=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
sip400=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0].iloc[:,1])
sip600=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1].iloc[:,1])
payload=np.append(sip500,sip400)
payload=np.append(payload,sip600)
payload=np.unique(payload)
payload=payload[payload!='BRK.B']
payload=payload[payload!='JW.A']
payload=payload[payload!='BF.B']

#print(payload)


class algo:
    start="2010-01-04"
    end='2021-12-04'
    interval='1wk'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        try:
            df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())
            df['stock']=self.stock
            #df['log_current']=np.log(df.Close/df.Open)
            df['log_current']=((df.Close-df.Open)/df.Open)
            #output = stock data from start to end
            clear_output(wait=False)
        except:
            pass
    
        return(df)
    
    
    def stockloop():
        l1=0
        l2=len(payload)
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
                print(i)
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
#dff.to_csv('raw.csv')
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))
print(dff)
#time.sleep(6000)

1500
Duration: 0:11:54.961745
                       Open       High        Low      Close  Adj Close  \
stock Date                                                                
A     2010-01-04  22.453505  22.625179  21.745352  22.031473  20.134977   
      2010-01-11  22.088697  22.432047  21.494993  21.766809  19.893097   
      2010-01-18  21.716738  22.253220  20.808298  20.865522  19.069399   
      2010-01-25  21.044350  21.208870  19.971388  20.050072  18.324139   
      2010-02-01  20.143063  21.380545  20.121601  20.865522  19.069399   
...                     ...        ...        ...        ...        ...   
ZYXI  2021-11-01  12.550000  14.500000  12.110000  14.070000  14.070000   
      2021-11-08  14.130000  14.910000  13.910000  14.600000  14.600000   
      2021-11-15  14.630000  15.660000  14.410000  14.580000  14.580000   
      2021-11-22  14.520000  14.630000  12.610000  12.860000  12.860000   
      2021-11-29  12.980000  13.335000  11.680000  11.850000  11.85000

In [8]:
start=(datetime.now())
from ta.volume import ChaikinMoneyFlowIndicator
from ta import add_all_ta_features
from ta.utils import dropna
#print(dff)
class algo2:
    dfmass=pd.DataFrame()
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        
        df=self.stock
        df['Open1']=df.Open.shift(1)
        df['Close1']=df.Close.shift(1)
        df['Low1']=df.Low.shift(1)
        df['High1']=df.High.shift(1)
        df['Volume1']=df.Volume.shift(1)
        df['Chng']=df.Close/df.Open-1
        df=df.dropna()
        df = add_all_ta_features(df, open="Open1", high="High1", low="Low1", close="Close1", volume="Volume1", fillna=True).dropna()
        
        #df=df.dropna()
        return(df)
        
    def slicer(self):
        df=self.stock
        #ar_perc_t=np.array([])
        #ar_perc_b=np.array([])
        ar_perc_diff=np.array([])
        df_t=df[df['Chng']>0]
        df_b=df[df['Chng']<0]
        for c in range(14,len(df.columns)):
            #print(df.columns[c])
            t_ave=np.average(df_t.iloc[:,c])
            b_ave=np.average(df_b.iloc[:,c])
            #l_ave=np.append(l_ave,t_ave)
            perc_t=stats.percentileofscore(df.iloc[:,c],t_ave)
            perc_b=stats.percentileofscore(df.iloc[:,c],b_ave)
            perc_diff=perc_t-perc_b
            #ar_perc_t=np.append(ar_perc_t,perc_t)
            #ar_perc_b=np.append(ar_perc_b,perc_b)
            ar_perc_diff=np.append(ar_perc_diff,perc_diff)
        #print(ar_perc)
        return(ar_perc_diff)
        #output = stock data with indicators
       


    def stockloop():
        boom=np.array([])
        for i in uniq:
            try:
                rawdata=(dff.loc[i])
                ta=algo2(rawdata).indicators()
                #print(ta)
                percs=algo2(ta).slicer()
                #print(percs)
                boom=np.append(boom,percs)
                #print(boom)
            except:
                pass
        #boom=boom.reshape(len(uniq),len(ta.columns)-9)
        #boom=pd.DataFrame(boom)
        #boom.columns=ta.columns[9:len(ta.columns)]
        #boom.index=uniq
        return(boom)
df_out=algo2.stockloop()
print(df_out)
#df_out.to_csv('percentiles.csv')
#print(arf)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

[-0.32206119 -1.77133655 -4.18679549 ... -0.1610306  -1.44927536
 -1.44927536]


In [9]:
print(len(df_out))
print(len(uniq))
dfw=df_out.reshape(1491,83)
dfw=pd.DataFrame(dfw)
dfw.to_csv('big_booba.csv')
print(dfw)

123753
1500
             0          1         2          3          4          5   \
0     -0.322061  -1.771337 -4.186795  -7.407407 -14.653784  -2.415459   
1      1.127214  -4.991948  4.508857  -8.534622  20.933977   5.797101   
2    -28.301887 -18.867925  5.660377  -3.773585  18.867925  -3.773585   
3     -0.161031   0.000000 -8.534622 -17.069243  -1.610306  -4.830918   
4     -3.220612  -2.415459 -0.322061  -3.703704 -17.713366  -9.983897   
...         ...        ...       ...        ...        ...        ...   
1486  -1.449275  -5.152979  2.093398   3.542673   7.085346  11.916264   
1487   0.000000  -0.483092 -0.322061  -6.280193 -11.272142  -2.576490   
1488   2.169197   1.301518 -4.989154 -12.364425 -10.845987  -1.952278   
1489  -5.797101  -5.636071  0.644122   0.483092  -2.093398  -1.288245   
1490  -0.644122  -7.568438  6.924316   1.127214   0.966184  -4.830918   

             6          7          8          9   ...        73         74  \
0     -0.483092   0.000000  -0.96